Este código implementa una GAN básica usando Keras para generar imágenes de dígitos similares a los de MNIST. El generador aprende a crear imágenes que se asemejan a las imágenes reales, mientras que el discriminador aprende a distinguir entre imágenes reales y falsas. Durante el entrenamiento, estos dos modelos se entrenan conjuntamente hasta que el generador produce imágenes realistas.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input, Model


#### Cargar y preprocesar los datos MNIST

In [2]:
# Cargar el conjunto de datos MNIST
(X_train, _), (_, _) = mnist.load_data()

# Normalizar los datos
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

#### Construir el generador

In [3]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))

    return model

#### Construir el discriminador

In [4]:
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

#### Compilar los modelos

In [5]:
img_shape = (28, 28, 1)
latent_dim = 100

# Crear y compilar el discriminador
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Crear el generador
generator = build_generator(latent_dim)

# El generador toma ruido como entrada y genera imágenes
z = Input(shape=(latent_dim,))
img = generator(z)

# Para el modelo combinado, solo entrenamos el generador
discriminator.trainable = False

# El discriminador toma imágenes generadas como entrada y determina su validez
valid = discriminator(img)

# Modelo combinado (stacked generador y discriminador)
# Entrenamos el generador para engañar al discriminador
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

#### Entrenar la GAN

In [6]:
def train(epochs, batch_size=128, save_interval=50):
    # Cargar los datos
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Etiquetas para imágenes reales y falsas
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Entrenar el discriminador

        # Seleccionar un conjunto aleatorio de imágenes reales
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Generar un conjunto de imágenes falsas
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Entrenar el discriminador
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Entrenar el generador

        # Generar un nuevo conjunto de ruido
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Entrenar el generador para engañar al discriminador
        g_loss = combined.train_on_batch(noise, valid)

        # Imprimir el progreso
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        # Guardar imágenes generadas a intervalos
        if epoch % save_interval == 0:
            save_imgs(epoch)

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Escalar las imágenes generadas de [-1, 1] a [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"mnist_{epoch}.png")
    plt.close()

# Entrenar la GAN
train(epochs=10000, batch_size=64, save_interval=200)

2/2 [==============================] - 0s 2ms/step
0 [D loss: 0.6823345124721527 | D accuracy: 39.84375] [G loss: 0.5419685244560242]
2/2 [==============================] - 0s 2ms/step
1 [D loss: 0.3832572177052498 | D accuracy: 64.84375] [G loss: 0.5553356409072876]
2/2 [==============================] - 0s 3ms/step
2 [D loss: 0.3581586489453912 | D accuracy: 74.21875] [G loss: 0.6032804250717163]
2/2 [==============================] - 0s 3ms/step
3 [D loss: 0.33757764659821987 | D accuracy: 82.03125] [G loss: 0.6733099818229675]
2/2 [==============================] - 0s 3ms/step
4 [D loss: 0.31389412423595786 | D accuracy: 87.5] [G loss: 0.7350281476974487]
2/2 [==============================] - 0s 2ms/step
5 [D loss: 0.2850230960175395 | D accuracy: 96.09375] [G loss: 0.8660100698471069]
2/2 [==============================] - 0s 2ms/step
6 [D loss: 0.24617281975224614 | D accuracy: 97.65625] [G loss: 0.9664332866668701]
2/2 [==============================] - 0s 2ms/step
7 [D loss: 0

In [8]:
generator.save("generador_MNIST.keras")

In [9]:
discriminator.save("discriminador_MNIST.keras")